In [ ]:
sc.install_pypi_package("scipy==1.1.0")
sc.install_pypi_package("Cython==0.29.33")
sc.install_pypi_package("scikit-learn==0.24.2")
sc.install_pypi_package("pandas==0.23.2")
sc.install_pypi_package("s3fs==0.4.2")
sc.install_pypi_package("boto3==1.26.59")
sc.install_pypi_package("joblib==1.2.0")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import groupby
from operator import itemgetter

import pandas as pd
import boto3
import os
import pickle
import tempfile
import joblib
import io

MongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Frequency") \
    .config("spark.cores.max", 6) \
    .config("spark.executor.cores", 6) \
    .config("spark.executor.memory", "36g") \
    .getOrCreate()

In [ ]:
df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("database", "coreEngine") \
    .option("collection", "PriceFeatures").load() \
    .drop("_id")

In [ ]:
hf = df.toPandas()

In [ ]:
boundary = 20230101

hf.updateDate = hf.updateDate.astype('int32')
mask_train = hf.updateDate <= boundary
df_train = hf.loc[mask_train,:].set_index(["stockCode", "stockFullName", "updateDate"])

df_train_event = df_train[["event"]]
df_train_set = df_train.drop(labels="event", axis=1)

pipeline : 데이터 변환및 모델 학습 단계를 하나의 파이썬 객체로 묶어주는 도구 <br>
MinMaxScaler : 최소~최대 사이 값으로 스케일링 <br>
class_weight="balanced" : 각 클래스에게 균형잡힌 가중치를 적용 => 클래스가 불균형할 때 사용 <br>

In [ ]:
model = make_pipeline(MinMaxScaler(), LogisticRegression(class_weight="balanced"))

In [ ]:
model.fit(df_train_set, df_train_event)

In [ ]:
s3 = boto3.client(
    "s3",
    region_name="ap-northeast-2",
    aws_access_key_id="xxxxxxxxxxxxxxxxxxxxxx",
    aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxx")

In [ ]:
"""
f = io.BytesIO()
joblib.dump(model, f)
f.seek(0)
s3.put_object(Bucket="penta-engine", Key="HighFrequencyModel.pkl", Body=f)
"""

In [ ]:
# score = model.score(df_train_set, df_train_event)
# params = model.get_params(deep=True)

# print(score)
# print(params)

In [ ]:
# model_coef = pd.DataFrame(model.steps[1][1].coef_)
# model_coef.columns = df_train_set.columns
# model_coef = model_coef.T.sort_values(by=0)
# model_coef.columns = ['coef']
# print(model_coef)